# Solar model - version 2

Date of creation: 17.06.2020

Last updated: 4.07.2020

In [2]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
h = 1.054E-27 # reduced Planck constant
c = 3e10 # speed of light
G = 6.67E-8 # gravitational constant
kB = 1.381E-16 # Boltzmann constant
m_prot = 1.67E-24 # масса протона
m_elec = 9.11E-28 # масса электрона
alpha = 137 # e2/h/c - постоянная тонкой структуры
e2 = h*c/alpha
r_elec = e2/m_elec/c/c # классический радиус электрона
pi = math.pi # 3.14...
sigmaT = 8*pi/3*r_elec*r_elec # сечение томсоновского рассеяния на электронах
kappaT = sigmaT/m_prot
sigma = pow(pi, 2)*pow(kB, 4)/60*pow(h, -3)*pow(c, -2) # sigma*T^4
a = pow(pi, 2)*pow(kB, 4)/15*pow(h, -3)*pow(c, -3)
gamma = 5/3

In [4]:
M_sol = 1.99e33 # solar mass
R_sol = 6.96e10 # solar radius
L_sol = 3.85e33 # solar luminosity
Teff = pow(L_sol/4/pi/sigma/R_sol/R_sol, 1/4) # 5780 solar surface temperature

## Функции для модели

In [5]:
# Эволюция содержания водорода
dimM = 200
dimT = 50
Xpre = 0.732
xmt = np.full((dimM, dimT), Xpre)
epoch = 0 # 0 - dimT-1 
year = 365.25*24*3600
eVolt = 1.6e-12 # ergs
step_t = 1e9*year # step by time
dEpp = (26.23 - 0.5)*1e6*eVolt

In [6]:
epoch = 0

In [7]:
# Интерполяция содержания водорода для данной массы
def XYZ(M, Z=0.02, dimM=200) :
    i = int(M*dimM)
    if i > dimM - 1 :
        i = dimM - 1
    X = xmt[i, epoch]
    return (X, 1-Z-X, Z)

In [8]:
# for i in range(100):
#     xmt[i, epoch] = 0.

In [9]:
xmt[:,epoch]

array([0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732, 0.732,
       0.732, 0.732,

In [10]:
def MU(X, Y, Z) :
    return  1/(2*X + 3/4*Y + 1/2*Z)

# Табличные данные для функции непрозрачности
Tst = [2120000.,  2720000.,  3400000.,  4640000.,  6030000.,
    7270000.,  8000000.,  8770000.,  9600000., 10200000., 10800000.,
   11400000., 12400000., 13800000., 14800000., 15500000.]
Kst = [9.94082840e+01, 3.60946746e+01,
   1.03492885e+01, 1.64912281e+00, 4.35865504e-01, 1.73913043e-01,
   1.08597285e-01, 7.14285714e-02, 4.93506494e-02, 3.93873085e-02,
   3.17164179e-02, 2.53164557e-02, 1.77439797e-02, 1.20259019e-02,
   8.96191187e-03, 7.03774792e-03]
def kappa(den, T, X, Y, Z) :
#     den *= 0.1
    if T <= Tst[0] :
#         return den*Kst[0]*T/Tst[0]
        return den*Kst[0]
    for i in range(len(Tst) - 1) :
        if T <= Tst[i+1] :
            return den*(Kst[i] + (T - Tst[i])/(Tst[i+1] - Tst[i])*(Kst[i+1] - Kst[i]))
    return den*Kst[-1]

In [11]:
# Уравнение состояния 
def Pressure(den, T, X, Y, Z) :
    mu = 1/(2*X + 3/4*Y + 1/2*Z)
    return den/m_prot/mu*kB*T

In [12]:
# Скорость энерговыделения в p-p реакции [эрг/с/г] деленная на X
def Epp(den, T, X) :
    T0 = (1e6, 5e6, 10e6, 15e6, 20e6, 30e6)
    e0 = (4e-9, 1.8e-3, 6.8e-2, 0.377, 1.09, 4.01)
    n = (10.6, 5.95, 4.60, 3.95, 3.64, 3.03)
    found = False
    for i in range (len(T0) - 1) :
        if T < pow(T0[i]*T0[i+1], 0.5) :
            found = True
            break;
        if not found :
            i = len(T0) - 1
    # print('i=%d T0=%.1e e0=%.3e n=%.2f' % (i, T0[i], e0[i], n[i]))
    return den*X*X*e0[i]*pow(T/T0[i], n[i])

# Скорость энерговыделения в CNO цикле [эрг/с/г]
def Ecno(den, T, X, XCNO) :
    T0 = (6e6, 10e6, 15e6, 20e6, 30e6, 50e6, 100e6)
    e0 = (9e-10, 3.4e-4, 1.94, 4.5e2, 4.1e5, 6.2e8, 1.9e12)
    n = (27.3, 22.9, 19.9, 18.0, 15.6, 13.6, 10.2)
    found = False
    for i in range (len(T0) - 1) :
        if T < pow(T0[i]*T0[i+1], 0.5) :
            found = True
            break;
    if not found :
        i = len(T0) - 1 
    # print('i=%d T0=%.1e e0=%.3e n=%.2f' % (i, T0[i], e0[i], n[i]))
    return den*X*XCNO*e0[i]*pow(T/T0[i], n[i])

# For best fit to SSM
def Etot(den, T, X, Y, Z) :
    return Epp(den, T, X) + Ecno(den, T, X, Z)
#     return 1.3*Epp(den, T, X) + 3.5*Ecno(den, T, X, Z)

## Интегрирование от центра по расстоянию
параметры модели: плотность и температура в центре.

хим.состав, число точек, шаг по радиусу.

In [23]:
def model_R(den0, T0, Z=0.02, dim=30000, dimM=200, step=R_sol/10000, debug=False) :
    dr = step  
    # distance from the center
    r = np.zeros(dim, dtype='float64')
    # density
    d = np.zeros(dim, dtype='float64')
    # temperature
    t = np.zeros(dim, dtype='float64')
    # светимость
    l = np.zeros(dim, dtype='float64')
    # mass inside r
    m = np.zeros(dim, dtype='float64')
    # pressure
    p = np.zeros(dim, dtype='float64')
    # hydrogen
    x = np.zeros(dim, dtype='float64')
    # opacity
    k = np.zeros(dim, dtype='float64')
    # 1 - convection, 0 - radiation
    conv = np.zeros(dim)
    # скорость выгорания водорода (еличина, обратная характерному времени)
    v = np.zeros(dim, dtype='float64')
    
# начальные данные
    
    # Step 1:
    for n in range(2) :
        r[n] = 10*n*dr
        d[n] = den0
        t[n] = T0
        V = 4*pi/3*pow(r[n], 3)
        m[n] = V*d[n]
        X, Y, Z = XYZ(m[n]/M_sol) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        x[n] = X
        mu = MU(X, Y, Z)
        l[n] = m[n]*Etot(d[n], t[n], X, Y, Z)
        p[n] = Pressure(d[n], t[n], X, Y, Z)
        k[n] = kappa(d[n], t[n], X, Y, Z)
        v[n] = 4*m_prot*Etot(d[n], t[n], X, Y, Z)/X/dEpp*year
# Ckeck for convection:
        Tgrad = k[n]*d[n]*l[n]/(4*a*c*pow(t[n], 3))
        cond = 8*pi/15*G*m_prot*mu/kB
        if debug :
            print(X, Y, Z, mu)
            print("Center: Tgrad=%.4e cond=%.4e" % (Tgrad, cond))
        if Tgrad > cond :
            conv[n] = 1
        else :
            conv[n] = 0
        if debug :
            print('Center: pressure=%e opacity=%.3f X=%.3f mu=%.5f' % (p[n], k[n], X, mu))
            if conv[n]:
                print("Convection")
            else:
                print("Radiation")
    warn = True
    for n in range(1, dim - 1) :
        X, Y, Z = XYZ(m[n]/M_sol)
        mu = MU(X, Y, Z)
        x[n] = X
        r[n+1] = r[n] + dr 
        S = 4*pi*r[n]*r[n]
        g = G*m[n]/r[n]/r[n]
        dp = -g*d[n]*dr
        p[n+1] = p[n] + dp
        k[n] = kappa(d[n], t[n], X, Y, Z)
        if debug :
            print("%d r=%.4f m=%.4f d=%.3f t=%.0f p=%.2e k=%.2f c=%.1f" % (n, r[n]/R_sol, m[n]/M_sol, d[n], t[n], p[n], k[n], conv[n]))

#             print('n=%d r/R_sol=%.4f m/M_sol=%.4f k=%.3f den=%.2f T=%.3e' % (n, r[n]/R_sol, m[n]/M_sol, k[n], d[n], t[n]))
        dt = -3/16*k[n]*d[n]*l[n]/sigma/pow(t[n], 3)/S*dr
        if dt < -2/5*g*m_prot*mu/kB*dr : # Условие конвекции
            dTconv = 2/5*t[n]/p[n]*dp
            if conv[n] == 0:
                if debug :
                    print("Begin convection zone at R=%.3f" % (r[n]/R_sol))
                    print("dT=%e porog=%e n=%d dtConv=%e" % (dt, -2/5*g*m_prot*mu/kB*dr, n, dTconv))
            conv[n+1] = 1
            dt = dTconv
        else :
            if conv[n] :
                if debug:
                    print("End convection zone at R=%.3f" % (r[n]/R_sol))
            conv[n+1] = 0
        
        t[n+1] = t[n] + dt
        # Находим плотность из уравнения состояния
        d[n+1] = mu*m_prot*p[n+1]/kB/t[n+1]
        davg = 0.5*(d[n] + d[n+1])
        tavg = 0.5*(t[n] + t[n+1])
        m[n+1] = m[n] + S*dr*davg
        l[n+1] = l[n] + S*dr*davg*Etot(davg, tavg, X, Y, Z)
        v[n] = 4*m_prot*Etot(d[n], t[n], X, Y, Z)/X/dEpp*year
        # уменьшаем шаг если температура падает слишком резко
        if abs(dt/t[n]) > 1e-3 :
            dr = 0.7*dr
#         if abs(dt/t[n]) < 0.5e-3 :
#             dr = 1.1*dr
        if m[n]/M_sol > 1.2 : # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            raise Exception('Mass exceeded 1.2*M_sol')
        Tsur = pow(l[n]/sigma/S, 1/4)
        if t[n] < Tsur :
            break
        if p[n] < 0. : 
#             print ('Negative Pressure')
            break

    if (n == dim - 2) :
         raise Exception("Error, increase dim!")
    
  # Проверка поверхностной температуры:
    if debug :
        Tsur = pow(l[n]/4/pi/sigma/r[n]/r[n], 1/4)
        print('Tsur=%.0f T=%.0f' % (Tsur, t[n]))

    df = pd.DataFrame({'Mass': m[:n], 'Radius': r[:n], 'Temperature': t[:n], 'Density': d[:n], 'Luminosity': l[:n], 
                       'Hydrogen': x[:n], 'Opacity': k[:n], 'Convection': conv[:n], 'Pressure': p[:n], 'Velocity': v[:n]})

#     mas = np.zeros(dimM)
#     rad = np.zeros(dimM)
#     tem = np.zeros(dimM)
#     den = np.zeros(dimM)
#     lum = np.zeros(dimM)
#     hyd = np.zeros(dimM)
#     opa = np.zeros(dimM)
#     pre = np.zeros(dimM)
#     con = np.zeros(dimM)
#     iprev = -1
#     for j in range(n+1) :
#         i = int(m[j]/m[n]*dimM)
#         if i == dimM :
#             i = dimM - 1
#         if i > iprev :
#             iprev = i
#             count = 0
#             sd = 0.
#             sl = 0.
#             sh = 0.
#             so = 0.
#             sp = 0.
#             sc = 0.
#         mas[i] = m[n]*(i+1)/dimM
#         rad[i] = r[j]
#         tem[i] = t[j]
#         count += 1
#         sd += d[j]
#         sl += l[j]
#         sh += x[j]
#         so += k[j]
#         sp += p[j]
#         sc += conv[j]
#         den[i] = sd/count
#         lum[i] = sl/count
#         hyd[i] = sh/count
#         opa[i] = so/count
#         pre[i] = sp/count
#         con[i] = sc/count
#     den[0] = d[0]
#     rad[0] = r[0]
#     tem[0] = t[0]
#     pre[0] = p[0]
#     tem[-1] = t[n]
#     rad[-1] = r[n]
#     df = pd.DataFrame({'Mass':mas, 'Radius':rad, 'Temperature':tem, 'Density':den, 'Luminosity':lum, 'Hydrogen':hyd, 'Opacity':opa, 'Convection':con})
    return (n, m[n], r[n], l[n], t[n], df)

In [21]:
%%time
T0 = (13.9)*1e6
den0 = 90
n, M, R, L, T, df = model_R(den0, T0, debug=False)


Exception: Mass exceeded 1.2*M_sol

In [15]:
print('T0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))

NameError: name 'n' is not defined

In [24]:
T0 = (13.31)*1e6
den0 = 95
n, M, R, L, T, df = model_R(den0, T0, debug=True)
n, M/M_sol, R/R_sol, L/L_sol, T

0.732 0.248 0.02 0.6024096385542169
Center: Tgrad=0.0000e+00 cond=8.1412e-16
Center: pressure=1.735749e+17 opacity=1.333 X=0.732 mu=0.60241
Radiation
0.732 0.248 0.02 0.6024096385542169
Center: Tgrad=9.6885e+10 cond=8.1412e-16
Center: pressure=1.735749e+17 opacity=1.333 X=0.732 mu=0.60241
Convection
1 r=0.0010 m=0.0000 d=95.000 t=13310000 p=1.74e+17 k=1.33 c=1.0
End convection zone at R=0.001
2 r=0.0011 m=0.0000 d=95.000 t=13309967 p=1.74e+17 k=1.33 c=0.0
3 r=0.0012 m=0.0000 d=94.999 t=13309931 p=1.74e+17 k=1.33 c=0.0
4 r=0.0013 m=0.0000 d=94.999 t=13309893 p=1.74e+17 k=1.33 c=0.0
5 r=0.0014 m=0.0000 d=94.998 t=13309851 p=1.74e+17 k=1.33 c=0.0
6 r=0.0015 m=0.0000 d=94.997 t=13309807 p=1.74e+17 k=1.33 c=0.0
7 r=0.0016 m=0.0000 d=94.997 t=13309760 p=1.74e+17 k=1.33 c=0.0
8 r=0.0017 m=0.0000 d=94.996 t=13309710 p=1.74e+17 k=1.33 c=0.0
9 r=0.0018 m=0.0000 d=94.996 t=13309657 p=1.74e+17 k=1.33 c=0.0
10 r=0.0019 m=0.0000 d=94.995 t=13309600 p=1.74e+17 k=1.33 c=0.0
11 r=0.0020 m=0.0000 d=94.9

795 r=0.0804 m=0.0320 d=81.938 t=12318246 p=1.39e+17 k=1.50 c=0.0
796 r=0.0805 m=0.0322 d=81.907 t=12315959 p=1.38e+17 k=1.51 c=0.0
797 r=0.0806 m=0.0323 d=81.877 t=12313669 p=1.38e+17 k=1.51 c=0.0
798 r=0.0807 m=0.0324 d=81.846 t=12311376 p=1.38e+17 k=1.51 c=0.0
799 r=0.0808 m=0.0325 d=81.815 t=12309081 p=1.38e+17 k=1.51 c=0.0
800 r=0.0809 m=0.0326 d=81.785 t=12306783 p=1.38e+17 k=1.51 c=0.0
801 r=0.0810 m=0.0327 d=81.754 t=12304483 p=1.38e+17 k=1.51 c=0.0
802 r=0.0811 m=0.0328 d=81.723 t=12302180 p=1.38e+17 k=1.51 c=0.0
803 r=0.0812 m=0.0330 d=81.692 t=12299874 p=1.38e+17 k=1.51 c=0.0
804 r=0.0813 m=0.0331 d=81.662 t=12297566 p=1.38e+17 k=1.51 c=0.0
805 r=0.0814 m=0.0332 d=81.631 t=12295255 p=1.38e+17 k=1.51 c=0.0
806 r=0.0815 m=0.0333 d=81.600 t=12292941 p=1.38e+17 k=1.51 c=0.0
807 r=0.0816 m=0.0334 d=81.569 t=12290625 p=1.38e+17 k=1.51 c=0.0
808 r=0.0817 m=0.0335 d=81.538 t=12288306 p=1.38e+17 k=1.52 c=0.0
809 r=0.0818 m=0.0336 d=81.507 t=12285985 p=1.37e+17 k=1.52 c=0.0
810 r=0.08

1329 r=0.1338 m=0.1269 d=63.085 t=10792880 p=9.35e+16 k=2.01 c=0.0
1330 r=0.1339 m=0.1271 d=63.047 t=10789568 p=9.34e+16 k=2.01 c=0.0
1331 r=0.1340 m=0.1274 d=63.009 t=10786255 p=9.33e+16 k=2.01 c=0.0
1332 r=0.1341 m=0.1276 d=62.971 t=10782940 p=9.32e+16 k=2.01 c=0.0
1333 r=0.1342 m=0.1278 d=62.933 t=10779623 p=9.31e+16 k=2.01 c=0.0
1334 r=0.1343 m=0.1281 d=62.894 t=10776304 p=9.30e+16 k=2.01 c=0.0
1335 r=0.1344 m=0.1283 d=62.856 t=10772983 p=9.30e+16 k=2.02 c=0.0
1336 r=0.1345 m=0.1286 d=62.818 t=10769661 p=9.29e+16 k=2.02 c=0.0
1337 r=0.1346 m=0.1288 d=62.780 t=10766337 p=9.28e+16 k=2.02 c=0.0
1338 r=0.1347 m=0.1290 d=62.742 t=10763011 p=9.27e+16 k=2.02 c=0.0
1339 r=0.1348 m=0.1293 d=62.703 t=10759684 p=9.26e+16 k=2.02 c=0.0
1340 r=0.1349 m=0.1295 d=62.665 t=10756354 p=9.25e+16 k=2.02 c=0.0
1341 r=0.1350 m=0.1298 d=62.627 t=10753023 p=9.24e+16 k=2.02 c=0.0
1342 r=0.1351 m=0.1300 d=62.589 t=10749690 p=9.24e+16 k=2.03 c=0.0
1343 r=0.1352 m=0.1303 d=62.550 t=10746356 p=9.23e+16 k=2.03 c

2009 r=0.2018 m=0.3284 d=38.672 t=8135002 p=4.32e+16 k=3.95 c=1.0
2010 r=0.2019 m=0.3288 d=38.640 t=8130520 p=4.31e+16 k=3.95 c=1.0
2011 r=0.2020 m=0.3291 d=38.608 t=8126039 p=4.31e+16 k=3.96 c=1.0
2012 r=0.2021 m=0.3294 d=38.576 t=8121556 p=4.30e+16 k=3.96 c=1.0
2013 r=0.2022 m=0.3298 d=38.544 t=8117074 p=4.29e+16 k=3.97 c=1.0
2014 r=0.2023 m=0.3301 d=38.512 t=8112592 p=4.29e+16 k=3.97 c=1.0
2015 r=0.2024 m=0.3304 d=38.480 t=8108109 p=4.28e+16 k=3.98 c=1.0
2016 r=0.2025 m=0.3308 d=38.448 t=8103627 p=4.28e+16 k=3.98 c=1.0
2017 r=0.2026 m=0.3311 d=38.416 t=8099144 p=4.27e+16 k=3.99 c=1.0
2018 r=0.2027 m=0.3315 d=38.384 t=8094661 p=4.27e+16 k=3.99 c=1.0
2019 r=0.2028 m=0.3318 d=38.352 t=8090178 p=4.26e+16 k=4.00 c=1.0
2020 r=0.2029 m=0.3321 d=38.320 t=8085695 p=4.25e+16 k=4.00 c=1.0
2021 r=0.2030 m=0.3325 d=38.289 t=8081212 p=4.25e+16 k=4.01 c=1.0
2022 r=0.2031 m=0.3328 d=38.257 t=8076729 p=4.24e+16 k=4.01 c=1.0
2023 r=0.2032 m=0.3331 d=38.225 t=8072245 p=4.24e+16 k=4.02 c=1.0
2024 r=0.2

2791 r=0.2800 m=0.5824 d=17.045 t=4713864 p=1.10e+16 k=27.01 c=1.0
2792 r=0.2801 m=0.5827 d=17.023 t=4709736 p=1.10e+16 k=27.04 c=1.0
2793 r=0.2802 m=0.5830 d=17.001 t=4705609 p=1.10e+16 k=27.06 c=1.0
2794 r=0.2803 m=0.5832 d=16.978 t=4701483 p=1.10e+16 k=27.09 c=1.0
2795 r=0.2804 m=0.5835 d=16.956 t=4697358 p=1.09e+16 k=27.11 c=1.0
2796 r=0.2805 m=0.5838 d=16.933 t=4693233 p=1.09e+16 k=27.14 c=1.0
2797 r=0.2806 m=0.5841 d=16.911 t=4689110 p=1.09e+16 k=27.16 c=1.0
2798 r=0.2807 m=0.5844 d=16.889 t=4684988 p=1.09e+16 k=27.19 c=1.0
2799 r=0.2808 m=0.5847 d=16.867 t=4680866 p=1.08e+16 k=27.21 c=1.0
2800 r=0.2809 m=0.5849 d=16.844 t=4676746 p=1.08e+16 k=27.24 c=1.0
2801 r=0.2810 m=0.5852 d=16.822 t=4672626 p=1.08e+16 k=27.26 c=1.0
2802 r=0.2811 m=0.5855 d=16.800 t=4668508 p=1.08e+16 k=27.29 c=1.0
2803 r=0.2812 m=0.5858 d=16.777 t=4664390 p=1.07e+16 k=27.31 c=1.0
2804 r=0.2813 m=0.5861 d=16.755 t=4660273 p=1.07e+16 k=27.33 c=1.0
2805 r=0.2814 m=0.5863 d=16.733 t=4656158 p=1.07e+16 k=27.36 c

...

8424 r=0.4219 m=0.7711 d=0.015 t=42983 p=8.69e+10 k=1.46 c=1.0
8425 r=0.4219 m=0.7711 d=0.015 t=42950 p=8.68e+10 k=1.46 c=1.0
8426 r=0.4219 m=0.7711 d=0.015 t=42917 p=8.66e+10 k=1.46 c=1.0
8427 r=0.4219 m=0.7711 d=0.015 t=42883 p=8.64e+10 k=1.46 c=1.0
8428 r=0.4219 m=0.7711 d=0.015 t=42850 p=8.63e+10 k=1.46 c=1.0
8429 r=0.4219 m=0.7711 d=0.015 t=42817 p=8.61e+10 k=1.46 c=1.0
8430 r=0.4219 m=0.7711 d=0.015 t=42783 p=8.59e+10 k=1.45 c=1.0
8431 r=0.4219 m=0.7711 d=0.015 t=42750 p=8.58e+10 k=1.45 c=1.0
8432 r=0.4219 m=0.7711 d=0.015 t=42717 p=8.56e+10 k=1.45 c=1.0
8433 r=0.4219 m=0.7711 d=0.015 t=42684 p=8.54e+10 k=1.45 c=1.0
8434 r=0.4219 m=0.7711 d=0.015 t=42650 p=8.53e+10 k=1.45 c=1.0
8435 r=0.4219 m=0.7711 d=0.015 t=42617 p=8.51e+10 k=1.45 c=1.0
8436 r=0.4219 m=0.7711 d=0.015 t=42584 p=8.49e+10 k=1.44 c=1.0
8437 r=0.4219 m=0.7711 d=0.015 t=42550 p=8.48e+10 k=1.44 c=1.0
8438 r=0.4219 m=0.7711 d=0.014 t=42517 p=8.46e+10 k=1.44 c=1.0
8439 r=0.4219 m=0.7711 d=0.014 t=42484 p=8.44e+10 k=1.4

9179 r=0.4228 m=0.7711 d=0.006 t=22695 p=1.76e+10 k=0.56 c=1.0
9180 r=0.4228 m=0.7711 d=0.006 t=22679 p=1.76e+10 k=0.56 c=1.0
9181 r=0.4228 m=0.7711 d=0.006 t=22662 p=1.75e+10 k=0.56 c=1.0
9182 r=0.4228 m=0.7711 d=0.006 t=22646 p=1.75e+10 k=0.56 c=1.0
9183 r=0.4228 m=0.7711 d=0.006 t=22630 p=1.75e+10 k=0.56 c=1.0
9184 r=0.4228 m=0.7711 d=0.006 t=22614 p=1.74e+10 k=0.56 c=1.0
9185 r=0.4228 m=0.7711 d=0.006 t=22597 p=1.74e+10 k=0.56 c=1.0
9186 r=0.4228 m=0.7711 d=0.006 t=22581 p=1.74e+10 k=0.56 c=1.0
9187 r=0.4228 m=0.7711 d=0.006 t=22565 p=1.73e+10 k=0.56 c=1.0
9188 r=0.4228 m=0.7711 d=0.006 t=22549 p=1.73e+10 k=0.56 c=1.0
9189 r=0.4228 m=0.7711 d=0.006 t=22532 p=1.73e+10 k=0.56 c=1.0
9190 r=0.4228 m=0.7711 d=0.006 t=22516 p=1.72e+10 k=0.55 c=1.0
9191 r=0.4228 m=0.7711 d=0.006 t=22500 p=1.72e+10 k=0.55 c=1.0
9192 r=0.4228 m=0.7711 d=0.006 t=22484 p=1.72e+10 k=0.55 c=1.0
9193 r=0.4228 m=0.7711 d=0.006 t=22467 p=1.72e+10 k=0.55 c=1.0
9194 r=0.4228 m=0.7711 d=0.006 t=22451 p=1.71e+10 k=0.5

9792 r=0.4231 m=0.7711 d=0.003 t=13779 p=5.05e+09 k=0.27 c=1.0
9793 r=0.4231 m=0.7711 d=0.003 t=13768 p=5.04e+09 k=0.27 c=1.0
9794 r=0.4231 m=0.7711 d=0.003 t=13757 p=5.03e+09 k=0.26 c=1.0
9795 r=0.4231 m=0.7711 d=0.003 t=13745 p=5.02e+09 k=0.26 c=1.0
9796 r=0.4231 m=0.7711 d=0.003 t=13734 p=5.01e+09 k=0.26 c=1.0
9797 r=0.4231 m=0.7711 d=0.003 t=13723 p=5.00e+09 k=0.26 c=1.0
9798 r=0.4231 m=0.7711 d=0.003 t=13711 p=4.99e+09 k=0.26 c=1.0
9799 r=0.4231 m=0.7711 d=0.003 t=13700 p=4.98e+09 k=0.26 c=1.0
9800 r=0.4231 m=0.7711 d=0.003 t=13689 p=4.97e+09 k=0.26 c=1.0
9801 r=0.4231 m=0.7711 d=0.003 t=13677 p=4.96e+09 k=0.26 c=1.0
9802 r=0.4231 m=0.7711 d=0.003 t=13666 p=4.95e+09 k=0.26 c=1.0
9803 r=0.4231 m=0.7711 d=0.003 t=13654 p=4.94e+09 k=0.26 c=1.0
9804 r=0.4231 m=0.7711 d=0.003 t=13643 p=4.92e+09 k=0.26 c=1.0
9805 r=0.4231 m=0.7711 d=0.003 t=13632 p=4.91e+09 k=0.26 c=1.0
9806 r=0.4231 m=0.7711 d=0.003 t=13620 p=4.90e+09 k=0.26 c=1.0
9807 r=0.4231 m=0.7711 d=0.003 t=13609 p=4.89e+09 k=0.2

10403 r=0.4234 m=0.7711 d=0.001 t=8190 p=1.37e+09 k=0.12 c=1.0
10404 r=0.4234 m=0.7711 d=0.001 t=8182 p=1.37e+09 k=0.12 c=1.0
10405 r=0.4234 m=0.7711 d=0.001 t=8174 p=1.37e+09 k=0.12 c=1.0
10406 r=0.4234 m=0.7711 d=0.001 t=8166 p=1.36e+09 k=0.12 c=1.0
10407 r=0.4234 m=0.7711 d=0.001 t=8158 p=1.36e+09 k=0.12 c=1.0
10408 r=0.4234 m=0.7711 d=0.001 t=8151 p=1.36e+09 k=0.12 c=1.0
10409 r=0.4234 m=0.7711 d=0.001 t=8143 p=1.35e+09 k=0.12 c=1.0
10410 r=0.4234 m=0.7711 d=0.001 t=8135 p=1.35e+09 k=0.12 c=1.0
10411 r=0.4234 m=0.7711 d=0.001 t=8127 p=1.35e+09 k=0.12 c=1.0
10412 r=0.4234 m=0.7711 d=0.001 t=8119 p=1.34e+09 k=0.12 c=1.0
10413 r=0.4234 m=0.7711 d=0.001 t=8111 p=1.34e+09 k=0.12 c=1.0
10414 r=0.4234 m=0.7711 d=0.001 t=8103 p=1.34e+09 k=0.12 c=1.0
10415 r=0.4234 m=0.7711 d=0.001 t=8095 p=1.33e+09 k=0.12 c=1.0
10416 r=0.4234 m=0.7711 d=0.001 t=8087 p=1.33e+09 k=0.12 c=1.0
10417 r=0.4234 m=0.7711 d=0.001 t=8079 p=1.33e+09 k=0.12 c=1.0
10418 r=0.4234 m=0.7711 d=0.001 t=8071 p=1.32e+09 k=0.1

(10463,
 0.7710539902552994,
 0.4233843163046257,
 0.591054804683255,
 7780.355597636594)

### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Первый вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации 
соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Внешний цикл по температурам, внутренний цикл - по плотности.
При фиксированной температуре постепенно повышаем плотность, пока не появится стабильное решение с массой < 1.
После этого возвращаемся на 1 шаг назад по плотности, уменьшаем шаг по плотности в 10 раз и повторям процедуру.
Так делаем пока шаг по плотности не станет меньше 1e-8.

In [27]:
%%time
dim = 1 # 11
for j in range(dim) :
    T0 = 1.333e7 + 0.001e7*j  # M = M_sol, epoch = 0
#     T0 = 13.9e6 + 0.1e6*j
    step = 10 # step by density
    exc = True
    den0 = 30
    if j > 0:
        df_prev = df.copy()
    while step > 1e-8 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                den0 += step
                break
            if exc :
                exc = False
                print('\tT0=%.5e D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                den0 -= step
                step /= 10.
                break
            
    print('# T0=%.5e D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1 :
#         df = df_prev
        break
print("Done!")

	T0=1.33300e+07 D0=100.0000000000 n=10194 M=0.6915 R=0.3877 L=0.5376 T=7945
	T0=1.33300e+07 D0=95.0000000000 n=10505 M=0.7835 R=0.4294 L=0.6033 T=7768
	T0=1.33300e+07 D0=94.6000000000 n=10992 M=0.8716 R=0.4935 L=0.6329 T=7331
	T0=1.33300e+07 D0=94.5600000000 n=11496 M=0.9267 R=0.5637 L=0.6382 T=6874
	T0=1.33300e+07 D0=94.5550000000 n=12147 M=0.9654 R=0.6552 L=0.6391 T=6381
	T0=1.33300e+07 D0=94.5545000000 n=12952 M=0.9885 R=0.7641 L=0.6392 T=5906
	T0=1.33300e+07 D0=94.5544800000 n=13258 M=0.9934 R=0.8063 L=0.6393 T=5752
	T0=1.33300e+07 D0=94.5544720000 n=13809 M=0.9987 R=0.8758 L=0.6393 T=5517
	T0=1.33300e+07 D0=94.5544714000 n=13970 M=0.9997 R=0.8961 L=0.6393 T=5454
	T0=1.33300e+07 D0=94.5544713300 n=14000 M=0.9999 R=0.8998 L=0.6393 T=5442
# T0=1.33300e+07 D0=94.55 n=14000 M=0.9999 R=0.8998 L=0.639 T=5442
Done!
Wall time: 1min 15s


In [28]:
# df = df_prev
df

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0.000000e+00,0.000000e+00,1.333000e+07,94.554471,0.000000e+00,0.732,1.318613,0.0,1.730205e+17,8.561869e-11
1,1.335360e+26,6.960000e+07,1.333000e+07,94.554471,1.634396e+27,0.732,1.318613,1.0,1.730205e+17,8.561869e-11
2,1.735967e+26,7.656000e+07,1.332997e+07,94.554040,2.124711e+27,0.732,1.318620,0.0,1.730193e+17,8.561741e-11
3,2.220700e+26,8.352000e+07,1.332993e+07,94.553578,2.717979e+27,0.732,1.318627,0.0,1.730180e+17,8.561603e-11
4,2.797568e+26,9.048000e+07,1.332990e+07,94.553080,3.424001e+27,0.732,1.318634,0.0,1.730166e+17,8.561455e-11
...,...,...,...,...,...,...,...,...,...,...
13995,1.989737e+33,6.262796e+10,5.465099e+03,0.000031,2.461129e+33,0.732,0.003113,0.0,2.349260e+07,4.899800e-49
13996,1.989737e+33,6.262800e+10,5.460450e+03,0.000031,2.461129e+33,0.732,0.003109,0.0,2.344258e+07,4.849587e-49
13997,1.989737e+33,6.262805e+10,5.455801e+03,0.000031,2.461129e+33,0.732,0.003105,0.0,2.339263e+07,4.799847e-49
13998,1.989737e+33,6.262810e+10,5.451153e+03,0.000031,2.461129e+33,0.732,0.003101,0.0,2.334274e+07,4.750576e-49


In [25]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/1epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

FromCenter/1epoch00.csv


### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Второй вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Альтернативный поиск (в обратном порядке)

Внешний цикл по плотности, внутренний цикл - по температуре. При фиксированной плотности постепенно понижаем температуру, пока не появится стабильное решение с массой < 1. После этого возвращаемся на 1 шаг назад по температуре, уменьшаем шаг по температуре в 10 раз и повторям процедуру. Так делаем пока шаг по температуре не станет меньше 0.01 градуса.

In [26]:
%%time
# Another order of fit - density outside
dim = 11
for j in range(dim) :
    den0 = 94 + 0.1*j # 94.4 + 0.1*j
    step = 1e6 # step for Temperature
    exc = True
    T0 = 15
    if j > 0:
        df_prev = df.copy()
    while step > 1e-2 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                T0 -= step
                break
            if exc :
                exc = False
                print('\tT0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                T0 += step
                step /= 10.
                break
            
    print('# T0=%.5f D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1. :
#         df = df_prev
        break
print("Done!")

	T0=15.00000 D0=94.0000000000 n=2 M=0.0000 R=0.0011 L=0.0000 T=15


KeyboardInterrupt: 

In [ ]:
# df = df_prev
df

In [ ]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/2epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

In [30]:
T0=1.33000e+07 
den0=94.4000000000
n, M, R, L, T, df = model_R(den0, T0, debug=False)
n, M/M_sol, R/R_sol, L/L_sol, T

(11049,
 0.8772552888605576,
 0.5009900889800694,
 0.6252592215835585,
 7252.863270233664)